In [1]:
import os
os.chdir(r"C:\Users\SGU\Projects\mlops-with-mlflow")
%pwd

'C:\\Users\\SGU\\Projects\\mlops-with-mlflow'

In [5]:
from mlProject.config.configuration import ConfigurationManager
import argparse
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib
import os
from dotenv import load_dotenv
load_dotenv()

model_name = "ElasticNet"
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))
mlflow.get_tracking_uri()
#exp = mlflow.get_experiment_by_name("Default")
runs = mlflow.search_runs(experiment_ids=["0"])
lowest_mse = runs["metrics.mean_squared_error"].sort_values(ascending=True)[0]
lowest_run_id = runs[runs["metrics.mean_squared_error"] == lowest_mse]["run_id"][0]
client = MlflowClient()
for mv in client.search_model_versions(f"name='{model_name}'"):
    print(mv)
    if mv.run_id == lowest_run_id:
        cur_version = mv.version
        logged_model = mv.source
        pprint(mv, indent=4)
        client.transition_model_version_stage(
            name=model_name,
            version=cur_version,
            stage="Production"
        )
loaded_model = mlflow.pyfunc.load_model(logged_model)
joblib.dump(loaded_model, os.path.join("artifacts", "prodmodel.joblib"))


    

[2023-08-12 14:29:40,358: DEBUG: connectionpool: https://dagshub.com:443 "POST /crazy-bot/mlops-with-mlflow.mlflow/api/2.0/mlflow/runs/search HTTP/1.1" 200 None]
[2023-08-12 14:29:40,564: DEBUG: connectionpool: https://dagshub.com:443 "GET /crazy-bot/mlops-with-mlflow.mlflow/api/2.0/mlflow/model-versions/search?filter=name%3D%27ElasticNet%27&max_results=10000 HTTP/1.1" 200 None]
<ModelVersion: aliases=[], creation_timestamp=1691837096537, current_stage='Production', description='', last_updated_timestamp=1691843201469, name='ElasticNet', run_id='72dd3a29b856452e81dc6cccf215df62', run_link='', source='mlflow-artifacts:/fc26881db9aa4a1f9db986531e56014f/72dd3a29b856452e81dc6cccf215df62/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='4'>
<ModelVersion: aliases=[], creation_timestamp=1691837096537, current_stage='Production', description='', last_updated_timestamp=1691843201469, name='ElasticNet', run_id='72dd3a29b856452e81dc6cccf215df62', run_link='', sou

['artifacts\\prodmodel.joblib']